In [3]:
import google.generativeai as genai

from dotenv import load_dotenv
load_dotenv() # load all environment variables from .env
import os

import mysql.connector
import ast

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

d:\signup\upwork\footbal\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#### Get Info using GenAI ####

def get_gemini_reponse(file, info_prompt, output):
    input_prompt = f'''
    You are an expert in understanding football match report. 
    You will receive input pdf as football match report and you will answer question
    based on the input pdf file.
    You should return {output}
    '''
    model = genai.GenerativeModel("gemini-1.5-flash")
    sample_pdf = genai.upload_file(file)
    response = model.generate_content([input_prompt, sample_pdf, info_prompt])
    return response.text
    


In [70]:
#### Save Info to Database #####
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="football"
)

cursor = db.cursor()

In [76]:
# cursor.execute('SHOW TABLES')


cursor.execute('CREATE DATABASE IF NOT EXISTS football;')
cursor.execute('USE football')
# cursor.execute('DROP DATABASE football;')

### NEED TO MODIFY VARIABLE FIRST BEFORE GO TO DATABASE
# cursor.execute('DROP TABLE Game')
# cursor.execute('TRUNCATE TABLE Game')


## Game

#### Get Info Game

In [8]:
game_info_input = 'match game information'
game_output = '''Get all match game information in JSON format. 

Use this JSON schema:
Game = {'championship': str, 
        'round': int, 
        'date': in YYYY Year-MM Month-DD day format, 
        'time': HH Hour:MM Minute:SS Second format, 
        'stadium': str, 
        'home_team': str, 
        'away_team': str, 
        'referee'; str, 
        'assistant_referee1': str, 
        'assistant_referee2'; str, 
        'fourth_official': str, 
        'result_1sthalf'; str, 
        'result_final': str}
without newline
'''

game_response = get_gemini_reponse('docs.pdf', game_info_input, game_output)
game_response
# game_list = json.loads(game_response)
# print(game_list)

"{'championship': 'Paulista - SUB20/2024', 'round': 21, 'date': '2024-09-08', 'time': '15:00', 'stadium': 'Estádio Municipal Hermínio Esposito / Embu das Artes', 'home_team': 'Referência FC', 'away_team': 'Desportivo Brasil', 'referee': 'Ronald Horns Araujo', 'assistant_referee1': 'Guilherme Holanda Moura Lima', 'assistant_referee2': 'Matheus Guilherme Biselli da Cruz', 'fourth_official': 'Diego Augusto Fagundes', 'result_1sthalf': '2 X 1', 'result_final': '2 X 1'}"

In [9]:
import ast

game = ast.literal_eval(game_response)
game

{'championship': 'Paulista - SUB20/2024',
 'round': 21,
 'date': '2024-09-08',
 'time': '15:00',
 'stadium': 'Estádio Municipal Hermínio Esposito / Embu das Artes',
 'home_team': 'Referência FC',
 'away_team': 'Desportivo Brasil',
 'referee': 'Ronald Horns Araujo',
 'assistant_referee1': 'Guilherme Holanda Moura Lima',
 'assistant_referee2': 'Matheus Guilherme Biselli da Cruz',
 'fourth_official': 'Diego Augusto Fagundes',
 'result_1sthalf': '2 X 1',
 'result_final': '2 X 1'}

#### Store Game in Mysql Database

In [77]:

# Create Table Game
cursor.execute('''CREATE TABLE IF NOT EXISTS GameS (
    game_id INT PRIMARY KEY AUTO_INCREMENT,
    championship VARCHAR(100),
    round smallint,
    date DATE,
    time TIME,
    stadium VARCHAR(150),
    home_team VARCHAR(50)  NOT NULL,
    away_team VARCHAR(50) NOT NULL,
    referee VARCHAR(50),
    assistant_referee1 VARCHAR(50),
    assistant_referee2 VARCHAR(50),
    fourth_official VARCHAR(50),
    result_1sthalf VARCHAR(10),
    result_final VARCHAR(10)
);''')
cursor.execute('ALTER TABLE gameS AUTO_INCREMENT=1000')


In [79]:
# Add Game Match
cursor.execute('''
    INSERT INTO Games (
        championship, round, date, time, stadium, home_team, away_team, referee,
        assistant_referee1, assistant_referee2, fourth_official, result_1sthalf, result_final
    ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
''', (game['championship'], game['round'], game['date'], game['time'], game['stadium'], 
    game['home_team'], game['away_team'], game['referee'], game['assistant_referee1'], 
    game['assistant_referee2'], game['fourth_official'], game['result_1sthalf'], 
    game['result_final']))
db.commit()

## Team

#### Get Info Team

In [78]:
team_info_input = 'team information'
team_output = '''Get all team information in JSON format. 

Use this JSON schema:
[{'team': str, 
        'type': either home/away}]
without newline
''' 

team_response = get_gemini_reponse('docs.pdf', team_info_input, team_output)
team_response

"[{'team': 'Referência FC', 'type': 'home'}, {'team': 'Desportivo Brasil', 'type': 'away'}]"

In [67]:
import ast

teams = ast.literal_eval(team_response)
teams

[{'team': 'Referência FC', 'type': 'home'},
 {'team': 'Desportivo Brasil', 'type': 'away'}]

In [68]:
for team in teams:
    print(team['team'])

Referência FC
Desportivo Brasil


#### Store Team in Mysql Database

In [ ]:
# cursor.execute('DROP TABLE Team')

In [80]:

# Create Table Team
cursor.execute('''CREATE TABLE IF NOT EXISTS Teams (
    team_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100),
    type VARCHAR(10),
    game_id INT,
    FOREIGN KEY(game_id) REFERENCES Games(game_id)
);''')
cursor.execute('ALTER TABLE Teams AUTO_INCREMENT=2000')


In [81]:
# Add Team
cursor.execute('SELECT game_id FROM games WHERE championship = %s', (game['championship'],))
game_id = cursor.fetchone()[0]  

for t in teams:
    cursor.execute('''
        INSERT INTO Teams (
            name, type, game_id
        ) 
        VALUES (%s, %s, %s)     ## must as String
    ''', (t['team'], t['type'], game_id))
    db.commit()

## Player

#### Get Info Player

In [44]:
player_info_input = 'player information'
player_output = '''Get all player information for each team in JSON format. 

Use this JSON schema:
[ {
    "team_name": "string",
    "players": [
        {
        'name': str either home/away, 
        'number': int,
        'registration': str,
        'position': either Starter or Reserve,
        'type': either Professional or Amateur,
        }
    ]
  } ]
without newline
'''

player_response = get_gemini_reponse('docs.pdf', player_info_input, player_output)
player_response

'[{"team_name": "Referência FC", "players": [{"name": "Gabriel Dias Braga", "number": 1, "registration": "544184/24", "position": "Starter", "type": "Amateur"}, {"name": "Francisco Ayello Guimarães", "number": 2, "registration": "544987/24", "position": "Starter", "type": "Amateur"}, {"name": "Marcelo das Neves Anastácio", "number": 3, "registration": "545578/24", "position": "Starter", "type": "Amateur"}, {"name": "Pedro Henrique Monteiro da Silva", "number": 4, "registration": "544186/24", "position": "Starter", "type": "Amateur"}, {"name": "Gabriel Xavier dos Santos", "number": 5, "registration": "572388/24", "position": "Starter", "type": "Amateur"}, {"name": "Guilherme Carlos Barrero Tonon", "number": 6, "registration": "550823/24", "position": "Starter", "type": "Amateur"}, {"name": "Pedro Henrique Duran Reis", "number": 7, "registration": "541211/24", "position": "Starter", "type": "Amateur"}, {"name": "Luciano Lima da Paz", "number": 8, "registration": "565324/24", "position": 

In [45]:
players = ast.literal_eval(player_response)
players

[{'team_name': 'Referência FC',
  'players': [{'name': 'Gabriel Dias Braga',
    'number': 1,
    'registration': '544184/24',
    'position': 'Starter',
    'type': 'Amateur'},
   {'name': 'Francisco Ayello Guimarães',
    'number': 2,
    'registration': '544987/24',
    'position': 'Starter',
    'type': 'Amateur'},
   {'name': 'Marcelo das Neves Anastácio',
    'number': 3,
    'registration': '545578/24',
    'position': 'Starter',
    'type': 'Amateur'},
   {'name': 'Pedro Henrique Monteiro da Silva',
    'number': 4,
    'registration': '544186/24',
    'position': 'Starter',
    'type': 'Amateur'},
   {'name': 'Gabriel Xavier dos Santos',
    'number': 5,
    'registration': '572388/24',
    'position': 'Starter',
    'type': 'Amateur'},
   {'name': 'Guilherme Carlos Barrero Tonon',
    'number': 6,
    'registration': '550823/24',
    'position': 'Starter',
    'type': 'Amateur'},
   {'name': 'Pedro Henrique Duran Reis',
    'number': 7,
    'registration': '541211/24',
    'p

In [46]:
for player in players:
    print(player)

{'team_name': 'Referência FC', 'players': [{'name': 'Gabriel Dias Braga', 'number': 1, 'registration': '544184/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Francisco Ayello Guimarães', 'number': 2, 'registration': '544987/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Marcelo das Neves Anastácio', 'number': 3, 'registration': '545578/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Pedro Henrique Monteiro da Silva', 'number': 4, 'registration': '544186/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Gabriel Xavier dos Santos', 'number': 5, 'registration': '572388/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Guilherme Carlos Barrero Tonon', 'number': 6, 'registration': '550823/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Pedro Henrique Duran Reis', 'number': 7, 'registration': '541211/24', 'position': 'Starter', 'type': 'Amateur'}, {'name': 'Luciano Lima da Paz', 'number': 8, 'registration': '565324/24', 'position': 'S

#### Store Player in Mysql Database

In [49]:
# Create Table Player
cursor.execute('''CREATE TABLE IF NOT EXISTS Players (
    player_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100),
    number smallint,
    registration VARCHAR(15),
    position VARCHAR(10),
    type VARCHAR(15),
    team_id INT,
    FOREIGN KEY(team_id) REFERENCES Teams(team_id)
);''')
cursor.execute('ALTER TABLE Players AUTO_INCREMENT=3000')


In [ ]:
# Add player
for team in players:
    cursor.execute('SELECT team_id FROM teams WHERE name = %s', (team['team_name'],))
    team_id = cursor.fetchone()[0]  
    for player in team['players']:
        cursor.execute('''
            INSERT INTO players (
                name, number, registration, position, type, team_id
            ) 
            VALUES (%s, %s, %s, %s, %s, %s)     ##  must as String
        ''', (player['name'], player['number'], player['registration'], player['position'], player['type'], team_id))
        db.commit()

##  Staff

In [47]:
staff_info_input = 'staff information'
staff_output = '''Get all staff information for each team in JSON format. 

Use this JSON schema:
[ {
    "team_name": "string",
    "staffs": [
        {
        'name': str, 
        'role': str
        }
    ]
  } ]
without newline
'''

staff_response = get_gemini_reponse('docs.pdf', staff_info_input, staff_output)
staff_response

'[{"team_name": "Referência FC", "staffs": [{"name": "Rodrigo de Oliveira Camargo", "role": "Técnico"}, {"name": "Danilo Claudino", "role": "Auxiliar Técnico"}, {"name": "Rafael Gobbato Brandão Cavalcanti", "role": "Médico"}, {"name": "Matheus Keller Vieira", "role": "Massagista"}, {"name": "Victor Staibano Gonçalves Wang", "role": "Preparador Físico"}]}, {"team_name": "Desportivo Brasil", "staffs": [{"name": "Fernando Lombardi", "role": "Técnico"}, {"name": "Jose Mario Soares do Prado", "role": "Auxiliar Técnico"}, {"name": "Fernando Cesar Siemann", "role": "Médico"}, {"name": "Misael Rodrigues de Oliveira", "role": "Massagista"}, {"name": "Luiz Fernando Bergamin", "role": "Preparador Físico"}]}]'

In [48]:
staffs = ast.literal_eval(staff_response)
staffs

[{'team_name': 'Referência FC',
  'staffs': [{'name': 'Rodrigo de Oliveira Camargo', 'role': 'Técnico'},
   {'name': 'Danilo Claudino', 'role': 'Auxiliar Técnico'},
   {'name': 'Rafael Gobbato Brandão Cavalcanti', 'role': 'Médico'},
   {'name': 'Matheus Keller Vieira', 'role': 'Massagista'},
   {'name': 'Victor Staibano Gonçalves Wang', 'role': 'Preparador Físico'}]},
 {'team_name': 'Desportivo Brasil',
  'staffs': [{'name': 'Fernando Lombardi', 'role': 'Técnico'},
   {'name': 'Jose Mario Soares do Prado', 'role': 'Auxiliar Técnico'},
   {'name': 'Fernando Cesar Siemann', 'role': 'Médico'},
   {'name': 'Misael Rodrigues de Oliveira', 'role': 'Massagista'},
   {'name': 'Luiz Fernando Bergamin', 'role': 'Preparador Físico'}]}]

In [58]:

# Create Table Staff
cursor.execute('''CREATE TABLE IF NOT EXISTS StaffS (
    staff_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100),
    role VARCHAR(20),
    team_id INT,
    FOREIGN KEY(team_id) REFERENCES Teams(team_id)
);''')
cursor.execute('ALTER TABLE StaffS AUTO_INCREMENT=4000')


In [59]:
# Add staff
for team in staffs:
    cursor.execute('SELECT team_id FROM teams WHERE name = %s', (team['team_name'],))
    team_id = cursor.fetchone()[0]  
    for staff in team['staffs']:
        cursor.execute('''
            INSERT INTO staffs (
                name, role, team_id
            ) 
            VALUES (%s, %s, %s)     ##  must as String
        ''', (staff['name'], staff['role'], team_id))
        db.commit()

## Substitution

In [49]:
substitution_info_input = 'substitution information'
substitution_output = '''Get all substitution information for each team in JSON format. 

Use this JSON schema:
[  {
    "team_name": "string",
    "substitutions": [
        {
        'player_out': int no of player, 
        'player_in': int no of player,
        'time': str,
        'half': str
'        }
    ]
  }]
without newline
'''

substitution_response = get_gemini_reponse('docs.pdf', substitution_info_input, substitution_output)
substitution_response

'[{"team_name": "Referência FC", "substitutions": [{"player_out": 5, "player_in": 15, "time": "INT", "half": "1T"}, {"player_out": 10, "player_in": 14, "time": "70:00", "half": "2T"}, {"player_out": 6, "player_in": 16, "time": "78:00", "half": "2T"}, {"player_out": 7, "player_in": 17, "time": "78:00", "half": "2T"}, {"player_out": 11, "player_in": 19, "time": "+ 2", "half": "2T"}]}, {"team_name": "Desportivo Brasil", "substitutions": [{"player_out": 8, "player_in": 19, "time": "56:00", "half": "2T"}, {"player_out": 4, "player_in": 13, "time": "56:00", "half": "2T"}, {"player_out": 2, "player_in": 15, "time": "66:00", "half": "2T"}, {"player_out": 5, "player_in": 20, "time": "85:00", "half": "2T"}]}]'

In [50]:
substitutions = ast.literal_eval(substitution_response)
substitutions

[{'team_name': 'Referência FC',
  'substitutions': [{'player_out': 5,
    'player_in': 15,
    'time': 'INT',
    'half': '1T'},
   {'player_out': 10, 'player_in': 14, 'time': '70:00', 'half': '2T'},
   {'player_out': 6, 'player_in': 16, 'time': '78:00', 'half': '2T'},
   {'player_out': 7, 'player_in': 17, 'time': '78:00', 'half': '2T'},
   {'player_out': 11, 'player_in': 19, 'time': '+ 2', 'half': '2T'}]},
 {'team_name': 'Desportivo Brasil',
  'substitutions': [{'player_out': 8,
    'player_in': 19,
    'time': '56:00',
    'half': '2T'},
   {'player_out': 4, 'player_in': 13, 'time': '56:00', 'half': '2T'},
   {'player_out': 2, 'player_in': 15, 'time': '66:00', 'half': '2T'},
   {'player_out': 5, 'player_in': 20, 'time': '85:00', 'half': '2T'}]}]

In [9]:
# Create Table Substitutions
cursor.execute('''CREATE TABLE IF NOT EXISTS Substitutions (
    substitution_id INT PRIMARY KEY AUTO_INCREMENT,
    player_out INT,
    player_in INT,
    time VARCHAR(6),
    team_id INT,
    half varchar(5),
    FOREIGN KEY(player_out) REFERENCES Players(player_id),
    FOREIGN KEY(player_in) REFERENCES Players(player_id),
    FOREIGN KEY(team_id) REFERENCES Teams(team_id)
);''')
cursor.execute('ALTER TABLE Substitutions AUTO_INCREMENT=5000')


In [ ]:
# Add substitution
for team in substitutions:
    cursor.execute('SELECT team_id FROM teams WHERE name = %s', (team['team_name'],))
    team_id = cursor.fetchone()[0]  
    for substitution in team['substitutions']:
        cursor.execute('SELECT player_id FROM Players WHERE number = %s and team_id=%s', (substitution['player_out'], team_id))
        player_out_id = cursor.fetchone()[0]
        
        cursor.execute('SELECT player_id FROM Players WHERE number = %s and team_id=%s', (substitution['player_in'], team_id))
        player_in_id = cursor.fetchone()[0]  
        
        cursor.execute('''
            INSERT INTO substitutions (
                player_out, player_in, time, team_id, half
            ) 
            VALUES (%s, %s, %s, %s, %s)     ##  must as String
        ''', (player_out_id, player_in_id, substitution['time'], team_id, substitution['half']))
        db.commit()

##  Goals

In [51]:
goal_info_input = 'goal information'
goal_output = '''Get all goal information for each team in JSON format. 

Use this JSON schema:
[  {
    "team_name": "string",
    "goals": [
        {
        'player_num': int no of player, 
        'time': str,
        'type':  "Penalty" (use full type names like "Normal", "Penalty", "Against", or "Foul")
'        }
    ]
  }]

without newline
'''

goal_response = get_gemini_reponse('docs.pdf', goal_info_input, goal_output)
goal_response

'[{"team_name": "Referência FC", "goals": [{"player_num": 11, "time": "25:00", "type": "Normal"}, {"player_num": 9, "time": "40:00", "type": "Normal"}]}, {"team_name": "Desportivo Brasil", "goals": [{"player_num": 3, "time": "10:00", "type": "Normal"}]}]'

In [52]:
goals = ast.literal_eval(goal_response)
goals

[{'team_name': 'Referência FC',
  'goals': [{'player_num': 11, 'time': '25:00', 'type': 'Normal'},
   {'player_num': 9, 'time': '40:00', 'type': 'Normal'}]},
 {'team_name': 'Desportivo Brasil',
  'goals': [{'player_num': 3, 'time': '10:00', 'type': 'Normal'}]}]

In [ ]:
# Create Table Goals
cursor.execute('''CREATE TABLE IF NOT EXISTS Goals (
    goal_id INT PRIMARY KEY AUTO_INCREMENT,
    player_id INT,
    time VARCHAR(5),
    team_id INT,
    type VARCHAR(10),
    FOREIGN KEY(player_id) REFERENCES Players(player_id),
    FOREIGN KEY(team_id) REFERENCES Teams(team_id)
);''')
cursor.execute('ALTER TABLE Goals AUTO_INCREMENT=6000')


In [ ]:
# Add Goals
for team in goals:
    cursor.execute('SELECT team_id FROM teams WHERE name = %s', (team['team_name'],))
    team_id = cursor.fetchone()[0]  
    for goal in team['goals']:
        cursor.execute('SELECT player_id FROM Players WHERE number = %s and team_id=%s', (goal['player_num'], team_id))
        player_id = cursor.fetchone()[0]
        
        cursor.execute('''
            INSERT INTO Goals (
                player_id, time, team_id, type
            ) 
            VALUES (%s, %s, %s, %s)     ##  must as String
        ''', (player_id, goal['time'], team_id, goal['type']))
        db.commit()

##  Cards

In [53]:
card_info_input = 'card information'
card_output = '''Get all card information for each team in JSON format. 

Use this JSON schema:
[  {
    "team_name": "string",
    "cards": [
        {
        'player_num': int no of player, 
        'time': str,
        'reason':  str,
        'half: str
'        }
    ]
  }]
without newline
'''

card_response = get_gemini_reponse('docs.pdf', card_info_input, card_output)
card_response

'[{"team_name": "Referência FC", "cards": [{"player_num": 8, "time": "58:00", "reason": "Impedir ataque promissor", "half": "2T"}, {"player_num": 16, "time": "80:00", "reason": "Impedir ataque promissor", "half": "2T"}]}, {"team_name": "Desportivo Brasil", "cards": [{"player_num": 8, "time": "51:00", "reason": "Ação temerária", "half": "2T"}, {"player_num": 15, "time": "68:00", "reason": "Ação temerária", "half": "2T"}]}]'

In [54]:
cards = ast.literal_eval(card_response)
cards

[{'team_name': 'Referência FC',
  'cards': [{'player_num': 8,
    'time': '58:00',
    'reason': 'Impedir ataque promissor',
    'half': '2T'},
   {'player_num': 16,
    'time': '80:00',
    'reason': 'Impedir ataque promissor',
    'half': '2T'}]},
 {'team_name': 'Desportivo Brasil',
  'cards': [{'player_num': 8,
    'time': '51:00',
    'reason': 'Ação temerária',
    'half': '2T'},
   {'player_num': 15,
    'time': '68:00',
    'reason': 'Ação temerária',
    'half': '2T'}]}]

In [30]:
# Create Table Card
cursor.execute('''CREATE TABLE IF NOT EXISTS Cards (
    card_id INT PRIMARY KEY AUTO_INCREMENT,
    player_id INT,
    time VARCHAR(5),
    reason VARCHAR(35),
    team_id INT,
    half VARCHAR(10),
    FOREIGN KEY(player_id) REFERENCES Players(player_id),
    FOREIGN KEY(team_id) REFERENCES Teams(team_id)
);''')
cursor.execute('ALTER TABLE Cards AUTO_INCREMENT=7000')


In [32]:
# Add Cards
for team in cards:
    cursor.execute('SELECT team_id FROM teams WHERE name = %s', (team['team_name'],))
    team_id = cursor.fetchone()[0]  
    for card in team['cards']:
        cursor.execute('SELECT player_id FROM Players WHERE number = %s and team_id=%s', (card['player_num'], team_id))
        player_id = cursor.fetchone()[0]
        
        cursor.execute('''
            INSERT INTO Cards (
                player_id, time, reason, team_id, half
            ) 
            VALUES (%s, %s, %s, %s, %s)     ##  must as String
        ''', (player_id, card['time'], card['reason'], team_id, card['half']))
        db.commit()

In [52]:
len('Leandro Roberto Parmigiani We love you')

38

# Test Modules

In [2]:
import google.generativeai as genai

from dotenv import load_dotenv
load_dotenv() # load all environment variables from .env
import os

import mysql.connector
import ast

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

d:\signup\upwork\footbal\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#### Save Info to Database #####
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root"
)

cursor = db.cursor()

In [15]:
cursor.execute('DROP DATABASE football')

In [16]:
from footballdb import extract_football_to_db

path = 'D:/signup/upwork/footbal/football_docs'
extract_football_to_db(path=path, db_user='root', db_password='root')
